In [1]:
import numpy as np
import cv2
import os 
import pandas as pd 
import pickle

In [2]:
face_det_model = "../models/res10_300x300_ssd_iter_140000.caffemodel"
face_det_proto = "../models/deploy.prototxt.txt"
face_descriptor = "../models/openface.nn4.small2.v1.t7"

detector = cv2.dnn.readNetFromCaffe(face_det_proto, face_det_model)
descriptor = cv2.dnn.readNetFromTorch(face_descriptor)

In [3]:
!dir "../emotion_data"

 Volume in drive C has no label.
 Volume Serial Number is 8E47-9470

 Directory of C:\Users\HP\Documents\COEP coding\Face recognition\5_Face_Recognition_MachineLearning\emotion_data

09-06-2021  11:59    <DIR>          .
09-06-2021  11:59    <DIR>          ..
09-06-2021  11:58    <DIR>          angry
09-06-2021  11:58    <DIR>          disgust
09-06-2021  11:58    <DIR>          fear
09-06-2021  11:58    <DIR>          happy
09-06-2021  11:59    <DIR>          neutral
09-06-2021  11:59    <DIR>          sad
09-06-2021  11:59    <DIR>          surprise
               0 File(s)              0 bytes
               9 Dir(s)  36,421,111,808 bytes free


In [4]:
img = cv2.imread("../emotion_data/surprise/1800.jpg")

In [5]:
def show(name, img):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
show("sample", img)

In [7]:
def helper(img_path):
    img = cv2.imread(img_path)
    image = img.copy()
    h, w = image.shape[:2]
    img_blob = cv2.dnn.blobFromImage(image, 1, (300, 300), (104, 177, 123), swapRB = False, crop = False)
    detector.setInput(img_blob)
    detections = detector.forward()

    if (len(detections) > 0):
        i = np.argmax(detections[0, 0, :, 2])
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (sx, sy, ex, ey) = box.astype('int')
    #         cv2.rectangle(image, (sx, sy), (ex, ey), (255, 0, 0))
            roi = image[sy:sx, ey:ex].copy()
            face_blob = cv2.dnn.blobFromImage(roi, 1/255, (96, 96), (0, 0, 0), swapRB = True, crop = True)
            descriptor.setInput(face_blob)
            vectors = descriptor.forward()
            return vectors
    return None

In [ ]:
helper("../emotion_data/surprise/1800.jpg")

In [33]:
show("Final", image)

In [9]:
data = dict(data = [], label = [])

In [10]:
folders = os.listdir("../emotion_data")
for folder in folders:
    files = os.listdir(f"../emotion_data/{folder}")
    for file in files:
        try:
            vector = helper(f"../emotion_data/{folder}/{file}")
            if vector is not None:
                data['data'].append(vector)
                data['label'].append(folder)
                print("Extracted !")
        except:
            pass

Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !
Extracted !


In [11]:
pd.Series(data['label']).value_counts()

sad         10
neutral      9
angry        7
happy        4
fear         3
surprise     1
dtype: int64

In [12]:
pickle.dump(data, open('data_pickle_emotion.pickle', mode='wb'))